<a href="https://colab.research.google.com/github/PauSanchoBeneyto/Mercadona_Pontia/blob/main/API_MERCADONAv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [233]:
import requests as req
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup as bs

In [234]:
#categoría
categorias='https://tienda.mercadona.es/api/categories/'
#tipo_categoria
url='https://tienda.mercadona.es/api/categories/65/'
#productos
productos='https://tienda.mercadona.es/api/products/'

In [235]:
#para ver el desglose en https://jsonviewer.stack.hu/
r_cat = req.get(categorias)
display(r_cat.text)

'{"next":null,"count":26,"results":[{"id":12,"name":"Aceite, especias y salsas","order":7,"layout":2,"published":true,"categories":[{"id":112,"name":"Aceite, vinagre y sal","order":7,"layout":1,"published":true,"is_extended":false},{"id":115,"name":"Especias","order":7,"layout":1,"published":true,"is_extended":false},{"id":116,"name":"Mayonesa, ketchup y mostaza","order":7,"layout":1,"published":true,"is_extended":false},{"id":117,"name":"Otras salsas","order":7,"layout":1,"published":true,"is_extended":false}],"is_extended":false},{"id":18,"name":"Agua y refrescos","order":8,"layout":2,"published":true,"categories":[{"id":156,"name":"Agua","order":8,"layout":1,"published":true,"is_extended":false},{"id":163,"name":"Isotónico y energético","order":8,"layout":1,"published":true,"is_extended":false},{"id":158,"name":"Refresco de cola","order":8,"layout":1,"published":true,"is_extended":false},{"id":159,"name":"Refresco de naranja y de limón","order":8,"layout":1,"published":true,"is_exte

In [236]:
r_url = req.get(url)
display(r_url.text)

'{"id":65,"name":"Bollería de horno","order":512,"layout":1,"published":true,"categories":[{"id":330,"name":"Bollería dulce","order":512,"layout":2,"products":[{"id":"84785","slug":"berlina-cacao-paquete","limit":999,"badges":{"is_water":false,"requires_age_check":false},"packaging":"Paquete","published":true,"share_url":"https://tienda.mercadona.es/product/84785/berlina-cacao-paquete","thumbnail":"https://prod-mercadona.imgix.net/images/59ea911cff1c1c0868030758825cc63b.jpg?fit=crop&h=300&w=300","categories":[{"id":5,"name":"Panadería y pastelería","level":0,"order":512}],"display_name":"Berlina al cacao","unavailable_from":null,"price_instructions":{"iva":10,"is_new":false,"is_pack":false,"pack_size":null,"unit_name":"ud.","unit_size":0.24,"bulk_price":"5.63","unit_price":"1.35","approx_size":false,"size_format":"kg","total_units":4,"unit_selector":true,"bunch_selector":false,"drained_weight":null,"selling_method":0,"price_decreased":false,"reference_price":"5.625","min_bunch_amount":

In [237]:
cat=r_cat.json()
url=r_url.json()
#prod =req.get(productos).json()

In [238]:
df_cat=pd.json_normalize(cat)
df_url=pd.json_normalize(url)

In [239]:
df_cat = pd.DataFrame(columns=['ID_cat', 'Nombre_categoria'])
df_pro = pd.DataFrame(columns=['ID_cat','subcategoria','ID_subcat','categoria_producto','ID_pro', 'Nombre', 'Precio'])
lista_subcategorias=[]

In [240]:
url = 'https://tienda.mercadona.es/api/categories/'
response = req.get(url)
lista_categorias = []
if response.status_code == 200:
    categorias = response.json()
    if 'results' in categorias:
        for categoria in categorias['results']:
            #print(f"ID {categoria['id']}: {categoria['name']}")
            cat=categoria['id']
            #OBTENEMOS EL LISTA DE  TODAS LAS CATEGORIAS PARA OBTENER LOS PRODUCTOS
            lista_categorias.append(cat)
            #Guardamos nuestra lista de productos con el id para poder concatenar en un futuro con los productos
            id=categoria['id']
            name_cat=categoria['name']
            catego = {'ID_cat': [id], 'Nombre_categoria': [name_cat]}
            df_2 = pd.DataFrame(catego)
            df_cat = pd.concat([df_cat,df_2])

In [241]:
df_pro = pd.DataFrame()
df=pd.DataFrame()

In [242]:
url = 'https://tienda.mercadona.es/api/categories/'
response = req.get(url)
lista_subcategorias = []

if response.status_code == 200:
    subcategorias = response.json()
    if 'results' in subcategorias:
        for categoria in subcategorias['results']:
            if 'categories' in categoria:
                for subcategoria in categoria['categories']:
                    id_sub = subcategoria['id']
                    name_subcat = subcategoria['name']
                    catego = {'ID_subcat': id_sub, 'subcategoria': name_subcat}
                    lista_subcategorias.append(id_sub)

# Crear DataFrame de las subcategorías
df_subcat = pd.DataFrame(lista_subcategorias)

In [243]:
for i in lista_subcategorias:
  url = "https://tienda.mercadona.es/api/categories/{}/"
  url=url.format(i)
  response = req.get(url)
  if response.status_code == 200:
      categoria = response.json()
      if 'categories' in categoria:
          for cat_info in categoria['categories']:
              if 'products' in cat_info:
                  for product in cat_info['products']:
                      product_id = product['id']
                      product_name = product['display_name']
                      product_price = product['price_instructions']['unit_price']
                      precio_KG_L = product['price_instructions']['bulk_price']
                      i = categoria['id']
                      c = categoria['name']
                      id_2=cat_info['id']
                      pro=cat_info['name']
                      datos = {'ID_cat': [i], 'subcategoria': [c], 'ID_subcat':[id_2],'categoria_producto':[pro],'ID_pro': [product_id],'Nombre':[product_name],'Precio':[product_price],'Precio_KG/L':[precio_KG_L]}
                      df = pd.DataFrame(datos)
                      df_pro = pd.concat([df_pro,df])


In [244]:
df = df_pro.reset_index(drop=True)

In [245]:
# Obtener la fecha actual
fecha_actual = datetime.today().strftime('%Y-%m-%d')  # Formato YYYY-MM-DD

# Agregar una nueva columna con la fecha actual al DataFrame
df['Fecha_actual'] = fecha_actual
df

,ID_cat,subcategoria,ID_subcat,categoria_producto,ID_pro,Nombre,Precio,Precio_KG/L,Fecha_actual
0,112,"Aceite, vinagre y sal",420,Aceite de oliva,4240,"Aceite de oliva 0,4º Hacendado",8.95,8.95,2024-05-07
1,112,"Aceite, vinagre y sal",420,Aceite de oliva,4717,Aceite de oliva virgen extra Hacendado,29.55,9.85,2024-05-07
2,112,"Aceite, vinagre y sal",420,Aceite de oliva,4740,Aceite de oliva virgen extra Hacendado,9.90,9.90,2024-05-07
3,112,"Aceite, vinagre y sal",420,Aceite de oliva,4706,Aceite de oliva virgen extra Hacendado Gran Se...,8.10,10.80,2024-05-07
4,112,"Aceite, vinagre y sal",420,Aceite de oliva,4640,Aceite de oliva 1º Hacendado,8.95,8.95,2024-05-07
...,...,...,...,...,...,...,...,...,...
5117,98,Tomate y otros sabores,101,Otros sabores,39643,Néctar de maracuyá Hacendado fruta de la pasió...,1.30,1.30,2024-05-07
5118,98,Tomate y otros sabores,101,Otros sabores,60588,Bebida de mango Hacendado sin azúcares añadidos,1.25,1.25,2024-05-07
5119,98,Tomate y otros sabores,101,Otros sabores,39604,Bebida de pomelo Hacendado sin azúcares añadidos,1.25,1.25,2024-05-07
5120,98,Tomate y otros sabores,101,Otros sabores,39922,Zumo de manzana selección Hacendado,1.50,1.50,2024-05-07


In [246]:
nombre_buscado = 'Zumo de manzana selección Hacendado'

fila = df.loc[df['Nombre'] == nombre_buscado]

if not fila.empty:
    print("Se encontró el nombre en el DataFrame:")
    print(fila)
else:
    print("El nombre no se encontró en el DataFrame.")

Se encontró el nombre en el DataFrame:
      ID_cat            subcategoria  ID_subcat categoria_producto ID_pro  \
5120      98  Tomate y otros sabores        101      Otros sabores  39922   

                                   Nombre Precio Precio_KG/L Fecha_actual  
5120  Zumo de manzana selección Hacendado   1.50        1.50   2024-05-07  


In [247]:
df.to_csv(r"Mercadona.csv",index=False)